<a href="https://colab.research.google.com/github/iamnileshmehta/Chatbot-projects/blob/main/IGP_Building_a_Conversational_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip

--2024-03-01 16:37:35--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2024-03-01 16:37:35--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [application/zip]
Saving to: ‘glove.twitter.27B.zip’

glove.twitter.27B.z 100%[===================>]   1.42G  4.99MB/s    in 4m 44s  

2024-03-01 16:42:19 (5.10 MB/s) - ‘glove.twitter.27B.zip’ saved [1520408563/1520408563]



In [ ]:
!pip3 install convokit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.2 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=7168fafdbdf5a9feeb915515a0447db411dae7fdf0ef54055205e8bb825ed1b3
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1

In [ ]:
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [ ]:
!unzip /content/glove.twitter.27B.zip

Archive:  /content/glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [ ]:
!wget http://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip

--2024-03-01 16:54:38--  http://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
Resolving zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)... 128.253.51.179
Connecting to zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)|128.253.51.179|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip [following]
--2024-03-01 16:54:38--  https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
Connecting to zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)|128.253.51.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40854701 (39M) [application/zip]
Saving to: ‘movie-corpus.zip’

movie-corpus.zip    100%[===================>]  38.96M  50.1MB/s    in 0.8s    

2024-03-01 16:54:39 (50.1 MB/s) - ‘movie-corpus.zip’ saved [40854701/40854701]



In [ ]:
!unzip /content/movie-corpus.zip

Archive:  /content/movie-corpus.zip
   creating: movie-corpus/
  inflating: movie-corpus/utterances.jsonl  
  inflating: movie-corpus/conversations.json  
  inflating: movie-corpus/corpus.json  
  inflating: movie-corpus/speakers.json  
  inflating: movie-corpus/index.json  


In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

# Load and preprocess the Cornell Movie Dialogue Corpus from JSON files
def load_dataset():
    # Load conversations from JSON file
    with open('/content/movie-corpus/conversations.json', 'r') as file:
        conversations_data = json.load(file)

    # Load lines from JSON file
    with open('/content/movie-corpus/speakers.json', 'r') as file:
        lines_data = json.load(file)

    # Extract conversations and lines
    conversations = conversations_data['conversations']
    lines = {line['lineID']: line['text'] for line in lines_data['lines']}

    # Create input-output pairs
    inputs, outputs = [], []
    for conv in conversations:
        for i in range(len(conv) - 1):
            inputs.append(lines[conv[i]])
            outputs.append(lines[conv[i+1]])

    return inputs, outputs

inputs, outputs = load_dataset()

# Tokenization and padding (similar to the previous code)
# Define the model (similar to the previous code)
# Compile and train the model (similar to the previous code)
# Inference model (similar to the previous code)
# Define inference function (similar to the previous code)
# Test the chatbot (similar to the previous code)


# Tokenization and padding
tokenizer_inputs = Tokenizer()
tokenizer_inputs.fit_on_texts(inputs)
inputs_sequences = tokenizer_inputs.texts_to_sequences(inputs)
max_len_input = max([len(seq) for seq in inputs_sequences])
inputs_sequences = pad_sequences(inputs_sequences, maxlen=max_len_input, padding='post')

tokenizer_outputs = Tokenizer()
tokenizer_outputs.fit_on_texts(outputs)
outputs_sequences = tokenizer_outputs.texts_to_sequences(outputs)
max_len_output = max([len(seq) for seq in outputs_sequences])
outputs_sequences = pad_sequences(outputs_sequences, maxlen=max_len_output, padding='post')

# Define the model
vocab_size_inputs = len(tokenizer_inputs.word_index) + 1
vocab_size_outputs = len(tokenizer_outputs.word_index) + 1
embedding_dim = 256
latent_dim = 512

encoder_inputs = tf.keras.Input(shape=(max_len_input,))
encoder_embedding = Embedding(vocab_size_inputs, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, dropout=0.5))(encoder_embedding)
encoder_lstm2 = LSTM(latent_dim, return_state=True, dropout=0.5)
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_lstm2)
state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.Input(shape=(None,))
decoder_embedding = Embedding(vocab_size_outputs, embedding_dim, mask_zero=True)
decoder_embedding = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, return_state=True, dropout=0.5)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_outputs, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit([inputs_sequences, outputs_sequences[:,:-1]], outputs_sequences[:,1:], batch_size=64, epochs=50, validation_split=0.2)

# Inference model
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

decoder_state_input_h = tf.keras.Input(shape=(latent_dim*2,))
decoder_state_input_c = tf.keras.Input(shape=(latent_dim*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = tf.keras.Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = tf.keras.Model([decoder_inputs_single] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Token index to word dictionary
reverse_word_index_inputs = {v: k for k, v in tokenizer_inputs.word_index.items()}
reverse_word_index_outputs = {v: k for k, v in tokenizer_outputs.word_index.items()}

# Define inference function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_outputs.word_index['<start>']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_word_index_outputs[sampled_token_index]
        if sampled_word != '<end>':
            decoded_sentence += ' ' + sampled_word
        if (sampled_word == '<end>' or len(decoded_sentence.split()) > max_len_output):
            stop_condition = True
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

# Test the chatbot
def chat(input_text):
    input_seq = tokenizer_inputs.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_len_input, padding='post')
    response = decode_sequence(input_seq)
    return response

# Example usage
print(chat("Hello"))


KeyError: 'conversations'

In [ ]:
def load_glove_embedding(file_path):
    embedding_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = [float(val) for val in values[1:]]
            embedding_dict[word] = vector
    return embedding_dict

glove_file_path = '/content/glove.twitter.27B.200d.txt'  # Adjust the path accordingly
glove_embedding = load_glove_embedding(glove_file_path)


In [ ]:
def filter_and_convert_conversations(conversations, max_word_length):
    filtered_conversations = []
    for conversation in conversations:
        filtered_dialogues = []
        for dialogue in conversation:
            if len(dialogue.split()) <= max_word_length:
                filtered_dialogues.append(dialogue)
        if len(filtered_dialogues) > 1:  # Ensure at least two dialogues for a conversation
            filtered_conversations.append(filtered_dialogues)

    input_texts = []
    target_texts = []

    for conversation in filtered_conversations:
        for i in range(len(conversation) - 1):
            input_text = '<start> ' + conversation[i]
            target_text = conversation[i + 1] + ' <end>'
            input_texts.append(input_text)
            target_texts.append(target_text)

    return input_texts, target_texts

# Example usage:
conversations = [
    ["Hello there!", "Hi, how are you?", "I'm good, thanks.", "That's great!"],
    ["Hey!", "Hey, what's up?", "Not much, just chilling.", "Cool."],
    ["Good morning!", "Morning, how did you sleep?", "Not too bad, thanks for asking.", "No problem."]
]

max_word_length = 5

input_texts, target_texts = filter_and_convert_conversations(conversations, max_word_length)

print("Input Texts:")
print(input_texts)
print("\nTarget Texts:")
print(target_texts)


In [ ]:
import numpy as np

def create_target_dictionaries(target_texts):
    target_word2id = {}
    target_id2word = {}
    for text in target_texts:
        for word in text.split():
            if word not in target_word2id:
                id = len(target_word2id)
                target_word2id[word] = id
                target_id2word[id] = word
    return target_word2id, target_id2word

# Example usage:
target_texts = [
    "how are you <end>",
    "I am fine <end>",
    "what about you <end>"
]

target_word2id, target_id2word = create_target_dictionaries(target_texts)

# Saving dictionaries as NumPy files
np.save('target_word2id.npy', target_word2id)
np.save('target_id2word.npy', target_id2word)


In [ ]:
import numpy as np

def prepare_input_data(sentences, word_embeddings, embedding_dim):
    input_data = []
    for sentence in sentences:
        sentence_embedding = []
        for word in sentence:
            if word in word_embeddings:
                word_embedding = word_embeddings[word]
            else:
                # If the word is not found in embeddings, use a zero vector
                word_embedding = np.zeros(embedding_dim)
            sentence_embedding.append(word_embedding)
        input_data.append(sentence_embedding)
    return input_data

# Example usage:
sentences = [
    ["This", "is", "a", "sample", "sentence"],
    ["Another", "sentence", "for", "demonstration"]
]

# Assuming 'glove_embedding' contains the GloVe embeddings loaded earlier
embedding_dim = len(glove_embedding['word'])  # Assuming all words have the same embedding dimension

input_data = prepare_input_data(sentences, glove_embedding, embedding_dim)


In [ ]:
def generate_batches(input_data, batch_size):
    num_batches = len(input_data) // batch_size
    batches = []
    for i in range(num_batches):
        start_index = i * batch_size
        end_index = start_index + batch_size
        batch = input_data[start_index:end_index]
        batches.append(batch)
    # Handling the last batch which may have fewer elements
    if len(input_data) % batch_size != 0:
        last_batch = input_data[num_batches * batch_size:]
        batches.append(last_batch)
    return batches

# Example usage:
batch_size = 2

batches = generate_batches(input_data, batch_size)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

def build_model(input_shape, target_vocab_size, latent_dim):
    # Define the encoder
    encoder_inputs = Input(shape=input_shape)
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    # Define the decoder
    decoder_inputs = Input(shape=(None, target_vocab_size))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

    # Dense layer to predict next token
    decoder_dense = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Compile the model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

    return model

# Example usage:
input_shape = (None, embedding_dim)  # Shape of input data
target_vocab_size = len(target_word2id)  # Size of the target vocabulary
latent_dim = 256  # Dimensionality of the latent space

model = build_model(input_shape, target_vocab_size, latent_dim)


In [ ]:
model.summary()

In [ ]:
# Assuming you have input_data and target_data prepared
# You need to reshape the target_data into one-hot encoding format

# Assuming you have the model trained and ready

# Generate predictions
predictions = model.predict([input_data, target_data])

# Now you have predictions for each input-output pair
